In [36]:

import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas import DataFrame

from bs4 import BeautifulSoup
import lxml

print('Libraries imported.')

Libraries imported.


In [37]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Download, scrape and wrangle

In [38]:
soup = BeautifulSoup(website_url.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

#get headers:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

#Find all items and skip first one:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

# skip all meta symbols and line feeds between rows:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

# make dataframe, expand rows and drop the old one:
postal_df=pd.DataFrame(rows)
postal_df[headers] = postal_df[0].str.split("</td>\n<td>", n = 2, expand = True) 
postal_df.drop(columns=[0],inplace=True)


## Steps performed in below cell : 

### 1.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
### 2. Areas with one postal code will be combined into one row with the neighborhoods separated with a comma.
### 3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [39]:
# skip not assigned boroughs:
postal_df = postal_df.drop(postal_df[(postal_df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
postal_df.Neighbourhood.replace("Not assigned", postal_df.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
postal_df.Neighbourhood.fillna(postal_df.Borough, inplace=True)
# drop duplicate rows:
postal_df=postal_df.drop_duplicates()

# extract titles from columns
postal_df.update(
    postal_df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

postal_df.update(
    postal_df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto annotation from Neighbourhood:
postal_df.update(
    postal_df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
postal_df.update(
    postal_df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

# combine multiple neighborhoods with the same post code
df2 = pd.DataFrame({'Postcode':postal_df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(postal_df['Borough'].loc[postal_df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(postal_df['Neighbourhood'].loc[postal_df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park


In [40]:
df2.head(15)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,"Scarborough, Toronto","Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [41]:
df2.shape

(103, 3)